# Concordância das Anotações de Entidades no Texto
Este notebook analisa a concordância dos anotadores em relação as entidades identificadas no texto

In [1]:
%%capture
%pip install spacy~=3.4.1

In [2]:
from spacy import tokens
from util import dataset, show
from util.annotation.category_filters import narrative

In [3]:
docs = dataset.convert()

In [4]:
len(docs)

357

In [5]:
print(sorted({d.user_data['week'] for d in docs}))

[1, 2, 3, 4, 5, 6, 7]


In [6]:
print(*docs[0].spans)

annot1 annot2


## Métricas Analisadas

`quantity_agreement`: Concordância média entre os anotadores, em relação a quantidade de elementos narrativos por texto.

`intersection_agreement`: Diante de todas as anotações que se intersectam, quantas têm o mesmo rótulo? (percentual)

Exemplificando métrica

In [7]:
from ..utils.annotation import comparison, analysis
import spacy
nlp = spacy.blank('pt')
doc = nlp('Eu vou para casa na segunda feira às 20:10. Já na sexta feira eu nem mesmo tenho aula. hahaha')
doc.spans['annot1'] = [
    tokens.Span(doc, 0, 1, 'narrador'), # Eu
    tokens.Span(doc, 0, 1, 'personagem'), # Eu < INTESECTA COM 'Eu' do annot2 mas não deve ser considerado
    tokens.Span(doc, 3, 4, 'lugar'), # casa
    tokens.Span(doc, 5, 9, 'tempo'), # segunda...
]
doc.spans['annot2'] = [
    tokens.Span(doc, 0, 1, 'narrador'), # Eu
    tokens.Span(doc, 3, 4, 'enredo'), # casa
    tokens.Span(doc, 5, 9, 'tempo'), # segunda...
    tokens.Span(doc, 12, 14, 'tempo'), # sexta... < DE FORA
]
show.span_pairs(*zip(*comparison.intersection(**doc.spans)))
print('-'*100)
show.span_pairs(*zip(*comparison.smart_intersection(**doc.spans)))
print('-'*100)
analysis.intersection_agreement([doc])

[00:01, narrador] 'Eu'                             | [00:01, narrador] 'Eu'                            
[00:01, personagem] 'Eu'                           | [00:01, narrador] 'Eu'                            
[03:04, lugar] 'casa'                              | [03:04, enredo] 'casa'                            
[05:09, tempo] 'segunda feira às 20:10'            | [05:09, tempo] 'segunda feira às 20:10'           
----------------------------------------------------------------------------------------------------
[00:01, narrador] 'Eu'                             | [00:01, narrador] 'Eu'                            
[03:04, lugar] 'casa'                              | [03:04, enredo] 'casa'                            
[05:09, tempo] 'segunda feira às 20:10'            | [05:09, tempo] 'segunda feira às 20:10'           
----------------------------------------------------------------------------------------------------


0.6666666666666666

Concordância média entre os anotadores, em relação a quantidade de elementos narrativos por texto.

In [8]:
analysis.quantity_agreement(docs)

0.0700280112044818

Diante de todas as anotações que se intersectam, quantas têm o mesmo rótulo? (média dos percentuais)


In [9]:
analysis.intersection_agreement(docs, filter_func=narrative)

0.9432404540763674

Caso 80

In [10]:
show.annotations(docs[80])
analysis.intersection_agreement([docs[80]], narrative)

-------------------annotator 1-------------------- | -------------------annotator 2--------------------
[09:10, Personagem] 'mulher'                       | [06:08, Organização te] 'Um dia'                  
[10:13, Ação] 'arcordou\ncedo'                     | [08:10, Personagem] 'Uma mulher'                  
[17:18, Lugar/Espaço] 'ônibus'                     | [08:13, Narrador] 'Uma mulher arcordou\ncedo'     
[41:42, Personagem] 'motorista'                    | [09:24, Ação] 'mulher arcordou\ncedo para que pega
[41:45, Ação] 'motorista saiu pra ver'             | [39:51, Ação] 'o\nmotorista saiu pra ver o que tin
[70:75, Ação] 'pessoas que estava\nandando'        | [61:69, Ação] 'O motorista vio muitas pegadas\nde 
[90:92, Personagem] 'homem esquesito'              | [81:86, Ação] 'ele foi vê quem estava'            
                                                   | [94:101, Ação] 'O homem mandou que todos\ndescesse
----------------------------------------------------------------

1

Caso 120

In [11]:
show.annotations(docs[120])
analysis.intersection_agreement([docs[120]], narrative)

-------------------annotator 1-------------------- | -------------------annotator 2--------------------
[13:14, Narrador] 'eu'                             | [10:13, Organização te] 'era una vez'             
[13:14, Personagem] 'eu'                           | [13:14, Narrador] 'eu'                            
[17:18, Personagem] 'aniga'                        | [13:14, Personagem] 'eu'                          
[18:23, Ação] 'brincando\nai OOV Pintar'           | [16:18, Personagem] 'ninha aniga'                 
[28:30, Ação] 'foi Viajar'                         | [26:30, Ação] 'agento \nfoi Viajar'               
[38:39, Lugar/Espaço] 'casa'                       | [41:61, Ação] 'eu cheguei Fui Pint\nai quando eu c
[43:45, Ação] 'Fui Pint'                           | [64:71, Ação] 'continuei a pintar ai eu durni ali'
[49:51, Ação] 'conesei apintar'                    | [73:79, Ação] 'acordei ai Fui to nar café'        
[64:67, Ação] 'continuei a pintar'                 |            

1

### Observando o caso do doc 118

In [12]:
show.annotations(docs[118])
analysis.intersection_agreement([docs[118]], narrative)

-------------------annotator 1-------------------- | -------------------annotator 2--------------------
[11:12, Narrador] 'Eu'                             | [11:12, Narrador] 'Eu'                            
[11:12, Personagem] 'Eu'                           | [11:12, Personagem] 'Eu'                          
[51:52, Lugar/Espaço] 'quarto'                     | [38:41, Ação] 'terei que repintar'                
[62:67, Ação] 'repintar\narrumar o quarto'         | [59:67, Ação] 'terei alem de repintar\narrumar o q
[69:70, Personagem] 'mãe'                          | [86:89, Ação] 'ela arrumando ele'                 
[86:88, Ação] 'ela arrumando'                      | [187:189, Ação] 'irei brincar'                    
[187:189, Ação] 'irei brincar'                     |                                                   
-------------------------------------------------------------------------------------------------------- 
     O sumiço das cores na parede e as roupas
Eu Quando voltei

1

Deveria ser 100%

In [13]:
show.annotations(docs[2], filter_func=narrative)
analysis.intersection_agreement(docs[2:3], filter_func=narrative)

-------------------annotator 1-------------------- | -------------------annotator 2--------------------
[01:02, Narrador] 'Eu'                             | [01:04, Narrador] 'Eu e Pedro'                    
[01:04, Personagem] 'Eu e Pedro'                   | [01:04, Personagem] 'Eu e Pedro'                  
[07:07, Lugar/Espaço] ''                           | [05:07, Lugar/Espaço] 'em casa'                   
[09:12, Ação] 'foi para escola'                    | [09:10, Ação] 'foi'                               
                                                   | [31:32, Ação] 'chegando'                          
                                                   | [48:49, Ação] 'foi'                               
                                                   | [52:53, Ação] 'guardo'                            
-------------------------------------------------------------------------------------------------------- 
 	Eu e Pedro estava em casa depois nos foi para escola
 ai dep

1